In [1]:
import pandas as pd 
import os
import json
import glob
import re
import sys
import concurrent.futures
sys.setrecursionlimit(20000) # Current limit = 999

from CommitGraph import CommitGraph

In [2]:
def createDirIfNotExists(folder_name):
    if not os.path.isdir(folder_name): 
        os.makedirs(folder_name)

In [3]:
root="/home/jovyan/work"
analysis_results_path = root + "/analysis/results/"
createDirIfNotExists(analysis_results_path)

In [39]:
def searchRegression(graph, init_node):
    candidates = []
    visited = []
    queue = []   

    visited.append(init_node)
    queue.append(init_node)

    while queue:
        node = queue.pop(0)  

        successParents = True
        parents = graph[node['commit']]['parents']

        for parent_hash in parents:
            if parent_hash not in graph: return [] # Reach final commit or experiment is limited
            parent = graph[parent_hash]
            successParent = parent['State'] == "TestSuccess"
            successParents = successParents and successParent
            if not successParent and parent_hash not in visited:
                if parent['State'] in ["BuildError", "TestBuildError"]:
                    candidates.append(node)
                if parent['State'] == "TestFail":
                    candidates = []
                visited.append(parent_hash)
                queue.append(parent)
    
        if successParents and node['State'] != "TestSuccess":
            candidates = candidates + [node]
            if len(parents) == 1 and node['State'] == 'TestFail': return [node]

    return candidates

In [5]:
def analyzeBug(project, bug_id):
    
    bug_name = "Bug_"+str(bug_id)
    bug_path = "{root}/results/{project}/{bug_name}/".format(root=root, project=project, bug_name=bug_name)
    
    with open("{root}/configFiles/{project}/bugs/{bug_name}.json".format(root=root,project=project, bug_name=bug_name)) as f:
        bug_info = json.load(f)
        
    test_name = re.search(r"-Dtest=(.*) test",bug_info['test_command']).group(1)

    bug_result = {
        'id': project + "_" + bug_name,
        'bug': bug_name,
        'project': project,
        "fix_pass": True,
        "prev_fails": True,
        "category": None,
        "sub_category": "-",
        "test_name": test_name,
        "bug_report": bug_info['bug_report'],
        "fix_commit": bug_info['fix_commit'],
        "BIC_candidates": []
    }
    
    if not os.path.isfile(bug_path+'commit_history.csv'):
        print(bug_path+'commit_history.csv')
        bug_result['category'] = "Other error"
        return bug_result
    
    results_dir = analysis_results_path+"{project}/{bug_name}/".format(project=project, bug_name=bug_name)
    createDirIfNotExists(results_dir)

    commit_graph = CommitGraph(project, bug_id, bug_path, results_dir, restore=True)
    
    fix_commit = commit_graph.graph[bug_info['fix_commit']]

    
    if not fix_commit['ExecuteTest']: # FIX COMMIT - SHOULD PASS
        
        bug_result['category'] = "Test fails in the fix commit"
        bug_result['fix_pass'] = False
        bug_result['prev_fails'] = None
        
        if fix_commit['Build']:
            if fix_commit['BuildTest']:
                if not fix_commit['HasTestReport']:
                    bug_result['sub_category'] = "The test was not executed"
                else:
                    bug_result['sub_category'] = "Test execution fails"
            else:
                bug_result['sub_category'] = "Failure in test build"
        else:
            bug_result['sub_category'] = "Failure in source build"
    
    else: 

        # SEARCH REGRESSION
        candidates = searchRegression(commit_graph.graph, fix_commit)
        if len(candidates) > 0:

            bug_result['BIC_candidates'] = list(map(lambda c: (c['id'],c['commit'] ),candidates))
            bug_result['category'] = "A regression is detected"

            if len(candidates) == 1:
                bug_result['sub_category'] = "Unique candidates" 
            else:
                bug_result['sub_category'] = "Multiple candidates" 

        # SEARCH OTHER BIC
        else:
            bug_result['category'] = "No regression is detected"
            bug_result['sub_category'] = "-"

#             success_test_build = [ r for r in results if r['BuildTest'] ]
#             candidate_commit = success_test_build[-1]
#             if candidate_commit['id']+1 == len(results):
#                 # Candidate commit is first commit
#                 bug_result['sub_category'] = "Reach first commit"
#             else:
#                 candidate_commit_antecesor = results[candidate_commit['id']+1]
#                 if candidate_commit_antecesor['Build']:
#                     bug_result['sub_category'] = "Failure in test build"
#                 else:
#                     bug_result['sub_category'] = "Failure in source build" 
#             bug_result['BIC_candidates'] = [(candidate_commit['commit'][0:8])]
#             bug_result['category'] = "No regression is detected"
    
    # Save bug result
    with open(results_dir+"bug_result.json",'w+') as json_file:
        json.dump(bug_result, json_file, indent=4)

    return bug_result

In [6]:
# CASE 1: Regression - Bug at commit 162
results = analyzeBug("JacksonCore", 11)['BIC_candidates']
results

[(162, '30a2670b336c3a4e4dee57460577b4bcc4b9e9b6')]

In [7]:
# CASE 2: No Regression - No candidate commit
results = analyzeBug("JacksonDatabind", 86)['BIC_candidates']
#results = analyzeBug("JacksonDatabind", 17)['BIC_candidates']
len(results)
results

[]

In [8]:
# CASE 3: Regresion but with buildability limitations
results = analyzeBug("JacksonDatabind", 38)['BIC_candidates']
len(results)

181

In [40]:
# CASE 4: Error at JacksonCore", 10
results = analyzeBug("JacksonCore", 10)['BIC_candidates']
len(results)

1

In [41]:
projects = [
    "JacksonXml", "Time", "Collections", "Compress", "Csv", "JacksonCore", "JacksonDatabind", "Gson", "Jsoup"
]
future_results = []
with concurrent.futures.ThreadPoolExecutor() as executor:
    for project in projects:
        # FOR EACH BUG
        for bug_path in glob.glob("{root}/results/{project}/Bug_*/".format(root=root, project=project)):
            bug = re.search(r"Bug_(\d+)", bug_path).group(1)
            try:
                future = executor.submit(analyzeBug, project, bug)
                future_results.append(future)
            except Exception as e:
                print(bug_path)
                print(e)
bug_results = [future.result() for future in future_results]

In [43]:
df = pd.DataFrame(bug_results)[['id', 'category', 'sub_category']]
df[df['category'] != None].set_index('id').sort_index()
print(len(df))
df['category'].value_counts()

348


No regression is detected       262
Test fails in the fix commit     47
A regression is detected         39
Name: category, dtype: int64

In [44]:
#df[df['category']=='Test failed at fix commit']
df.groupby(['category', "sub_category"]).count()['id']

category                      sub_category             
A regression is detected      Multiple candidates           17
                              Unique candidates             22
No regression is detected     -                            262
Test fails in the fix commit  Failure in source build       22
                              Failure in test build          3
                              Test execution fails           6
                              The test was not executed     16
Name: id, dtype: int64

In [20]:
df_projects = pd.DataFrame(bug_results)[['id', 'category', 'sub_category', 'project']]
df_projects[df_projects['project']=="Jsoup"].groupby(['category', "sub_category"]).count()['id']

category                      sub_category        
A regression is detected      Unique candidates        8
No regression is detected     -                       83
Test fails in the fix commit  Test execution fails     2
Name: id, dtype: int64

In [45]:
df[df['category']=='A regression is detected'][df['sub_category']=='Unique candidates']

/tmp/ipykernel_1712718/3306249005.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df['category']=='A regression is detected'][df['sub_category']=='Unique candidates']


,id,category,sub_category
9,Time_Bug_1,A regression is detected,Unique candidates
37,Compress_Bug_28,A regression is detected,Unique candidates
72,Compress_Bug_45,A regression is detected,Unique candidates
106,JacksonCore_Bug_11,A regression is detected,Unique candidates
115,JacksonCore_Bug_10,A regression is detected,Unique candidates
118,JacksonCore_Bug_21,A regression is detected,Unique candidates
154,JacksonDatabind_Bug_24,A regression is detected,Unique candidates
172,JacksonDatabind_Bug_35,A regression is detected,Unique candidates
180,JacksonDatabind_Bug_41,A regression is detected,Unique candidates
201,JacksonDatabind_Bug_87,A regression is detected,Unique candidates
